The cells here were not necessarily run in order towards the end as I would run cell 2 to 'reset' the initial grid for verification purposes along the way.

Steps:
1. Gaussian elimination was used to figure out which lights needed to be toggled.
2. A backtracking search algorithm over the gaussian eliminatino solution vector was performed and then terminated at a length it seemed to get 'stuck' at.
3. The remaining steps that the backtracking algorithm could not do were logically worked out by hand, hence the excessive amount of cell blocks with single toggles.

In [1]:
import numpy as np

def switch(grid, row, col):
    grid[row, :] ^= True
    grid[:, col] ^= True
    grid[row, col] ^= True

def changeIndex(i,j,s):
    return (j+1, s-i)

In [2]:
grid_24x24 = '''
    000001000000000001110011
    110100010110101000010011
    011101110000001101001110
    000110111000110101101100
    101101011010010011101010
    111000100101110100101000
    110001011100000000000101
    100000010001100000000010
    000110010010110110101001
    011101101011111011100000
    011000101010111011111100
    100011110010000100100111
    000111010010100010001110
    011001010001001111110101
    110001000010111000100000
    000000101100101000101001
    111001010010010011110110
    100000110001111111011010
    110100000011100100110010
    101000110111001110010000
    110000000010011100100101
    111111011011111100010101
    000000000110101011100000
    110001111100000011001111
    '''

rows = grid_24x24.strip().split('\n')
grid_24x24 = np.array([list(map(int, row.strip())) for row in rows], dtype=int)

In [3]:
def lights_out(grid):
    n = len(grid)
    A = np.zeros((n**2, n**2), dtype=int)  
    b = np.ones(n**2, dtype=int)         

    for i in range(n):
        for j in range(n):
            k = i * n + j
            A[k, k] = 1 
            for l in range(n):
                A[k, i*n+l] = 1
                A[k, l*n+j] = 1

    b *= np.array(grid).flatten()

    for j in range(n**2):
        i = j
        while i < n**2 and A[i, j] == 0:
            i += 1
        if i == n**2:
            continue  

        A[[i, j], :] = A[[j, i], :]
        b[[i, j]] = b[[j, i]]

        for i in range(j+1, n**2):
            if A[i, j] == 1:
                A[i, :] = (A[i, :] + A[j, :]) % 2
                b[i] = (b[i] + b[j]) % 2

    x = np.zeros(n**2, dtype=int)
    for j in range(n**2-1, -1, -1):
        if A[j, j] == 1:
            x[j] = b[j]
            for i in range(j):
                b[i] = (b[i] + A[i, j] * x[j]) % 2

    solution = np.logical_not(x.reshape((n, n))).astype(int).tolist()
    return solution

solution = lights_out(grid_24x24)

In [8]:
def solve(grid, bulbs, path):
    if len(bulbs)==8:
        print(path)
        return True
    for idx,(i,j) in enumerate(bulbs):
        if not grid[i][j]:
            switch(grid,i,j)
            bulbs.pop(idx)
            path.append((i,j))
            if solve(grid,bulbs,path):
                return True
            switch(grid,i,j)
            bulbs.append((i,j))
            path.pop()
    return False

bulbs = [(i,j) for (i,j),e in np.ndenumerate(solution) if e]
solve(grid_24x24, bulbs, [])

[(0, 0), (0, 17), (0, 6), (0, 18), (0, 8), (0, 19), (0, 10), (0, 22), (0, 11), (1, 0), (1, 12), (0, 12), (0, 14), (1, 2), (1, 17), (1, 4), (1, 18), (1, 10), (1, 23), (1, 13), (2, 0), (2, 1), (2, 2), (2, 3), (2, 10), (2, 7), (2, 12), (2, 8), (2, 22), (2, 11), (3, 3), (3, 0), (3, 4), (3, 10), (3, 5), (3, 14), (1, 14), (1, 15), (2, 15), (2, 17), (3, 7), (3, 15), (3, 11), (3, 18), (3, 13), (3, 21), (1, 21), (1, 16), (3, 17), (3, 23), (2, 23), (2, 21), (4, 4), (4, 0), (4, 5), (4, 1), (4, 9), (4, 8), (5, 0), (5, 4), (5, 3), (5, 16), (5, 6), (5, 19), (4, 19), (4, 10), (5, 7), (5, 20), (0, 20), (5, 11), (5, 21), (4, 21), (4, 15), (5, 12), (6, 6), (6, 1), (6, 7), (6, 10), (6, 9), (6, 14), (6, 11), (7, 2), (7, 0), (7, 4), (7, 1), (7, 9), (7, 3), (7, 11), (7, 12), (6, 12), (6, 20), (4, 20), (7, 13), (7, 18), (7, 15), (7, 19), (7, 16), (7, 21), (6, 21), (7, 17), (5, 17), (5, 22), (8, 1), (8, 2), (8, 17), (8, 9), (8, 19), (8, 10), (8, 23), (8, 12), (9, 1), (9, 0), (9, 13), (9, 2), (9, 16), (9, 3), 

True

In [9]:
path = [(0, 0), (0, 17), (0, 6), (0, 18), (0, 8), (0, 19), (0, 10), (0, 22), (0, 11), (1, 0), (1, 12), (0, 12), (0, 14), (1, 2), (1, 17), (1, 4), (1, 18), (1, 10), (1, 23), (1, 13), (2, 0), (2, 1), (2, 2), (2, 3), (2, 10), (2, 7), (2, 12), (2, 8), (2, 22), (2, 11), (3, 3), (3, 0), (3, 4), (3, 10), (3, 5), (3, 14), (1, 14), (1, 15), (2, 15), (2, 17), (3, 7), (3, 15), (3, 11), (3, 18), (3, 13), (3, 21), (1, 21), (1, 16), (3, 17), (3, 23), (2, 23), (2, 21), (4, 4), (4, 0), (4, 5), (4, 1), (4, 9), (4, 8), (5, 0), (5, 4), (5, 3), (5, 16), (5, 6), (5, 19), (4, 19), (4, 10), (5, 7), (5, 20), (0, 20), (5, 11), (5, 21), (4, 21), (4, 15), (5, 12), (6, 6), (6, 1), (6, 7), (6, 10), (6, 9), (6, 14), (6, 11), (7, 2), (7, 0), (7, 4), (7, 1), (7, 9), (7, 3), (7, 11), (7, 12), (6, 12), (6, 20), (4, 20), (7, 13), (7, 18), (7, 15), (7, 19), (7, 16), (7, 21), (6, 21), (7, 17), (5, 17), (5, 22), (8, 1), (8, 2), (8, 17), (8, 9), (8, 19), (8, 10), (8, 23), (8, 12), (9, 1), (9, 0), (9, 13), (9, 2), (9, 16), (9, 3), (9, 17), (9, 18), (9, 20), (8, 20), (8, 21), (10, 0), (10, 1), (10, 2), (10, 13), (10, 5), (10, 16), (10, 11), (10, 17), (10, 18), (10, 21), (10, 19), (11, 0), (11, 1), (11, 2), (11, 3), (11, 5), (11, 13), (11, 6), (11, 16), (11, 9), (11, 17), (11, 10), (11, 23), (9, 23), (11, 19), (12, 0), (12, 3), (12, 4), (12, 7), (12, 6), (12, 8), (12, 14), (12, 11), (12, 21), (12, 16), (12, 22), (8, 22), (13, 4), (13, 3), (13, 13), (13, 5), (13, 14), (13, 9), (14, 8), (14, 1), (14, 11), (13, 11), (13, 22), (14, 6), (14, 13), (14, 18), (14, 20), (15, 10), (15, 0), (15, 11), (15, 5), (15, 18), (15, 9), (16, 3), (16, 0), (16, 10), (16, 4), (17, 0), (17, 1), (17, 4), (17, 2), (17, 11), (17, 3), (17, 12), (17, 6), (17, 18), (17, 9), (16, 9), (16, 5), (17, 20), (17, 14), (17, 21), (18, 15), (16, 15), (18, 1), (18, 18), (18, 3), (18, 19), (18, 5), (18, 20), (18, 6), (18, 23), (10, 23), (18, 8), (19, 13), (19, 0), (19, 14), (18, 14), (18, 22), (19, 1), (19, 17), (19, 3), (19, 18), (19, 4), (20, 0), (20, 7), (20, 2), (20, 9), (20, 3), (20, 10), (19, 10), (19, 6), (20, 4), (20, 16), (20, 14), (20, 17), (20, 19), (20, 22), (19, 22), (19, 11), (21, 9), (21, 0), (21, 11), (21, 5), (21, 16), (21, 8), (21, 17), (21, 10), (22, 1), (22, 2), (22, 7), (22, 3), (22, 10), (22, 4), (23, 0), (23, 13), (22, 13), (22, 12), (21, 12), (21, 14), (22, 14), (22, 15), (23, 2), (23, 18), (21, 18), (21, 22), (22, 22), (23, 20), (22, 21), (19, 21), (23, 19), (23, 4), (23, 23), (12, 23), (23, 5), (22, 19)]

In [11]:
for (i,j) in path:
    switch(grid_24x24,i,j)

for row in grid_24x24:
    print(row)

for (i,j),e in np.ndenumerate(solution):
    if e and (i,j) not in path:
        print(i,j)

[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
[0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]


In [12]:
switch(grid_24x24,22,3)
for row in grid_24x24:
    print(row)

[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]


In [14]:
switch(grid_24x24,23,3)

for row in grid_24x24:
    print(row)

[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
[0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]


In [15]:
switch(grid_24x24,22,3)
for row in grid_24x24:
    print(row)

[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]


In [16]:
switch(grid_24x24,23,6)
for idx,row in enumerate(grid_24x24):
    print(idx,row)

0 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
1 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
2 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
3 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
4 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
5 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
6 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
7 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
8 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
9 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
10 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
11 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
12 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
13 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
14 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
15 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
16 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
17 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
18 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
19 

In [17]:
switch(grid_24x24,22,8)
for idx,row in enumerate(grid_24x24):
    print(idx,row)

0 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
1 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
2 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
3 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
4 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
5 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
6 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
7 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
8 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
9 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
10 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
11 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
12 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
13 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
14 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
15 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
16 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
17 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
18 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
19 

In [18]:
switch(grid_24x24,23,8)
for idx,row in enumerate(grid_24x24):
    print(idx,row)

0 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
1 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
2 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
3 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
4 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
5 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
6 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
7 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
8 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
9 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
10 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
11 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
12 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
13 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
14 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
15 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
16 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
17 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
18 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
19 

In [19]:
switch(grid_24x24,22,8)
for idx,row in enumerate(grid_24x24):
    print(idx,row)

0 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
1 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
2 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
3 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
4 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
5 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
6 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
7 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
8 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
9 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
10 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
11 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
12 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
13 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
14 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
15 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
16 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
17 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
18 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
19 

In [20]:
switch(grid_24x24,23,15)
for idx,row in enumerate(grid_24x24):
    print(idx,row)

0 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
1 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
2 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
3 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
4 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
5 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
6 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
7 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
8 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
9 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
10 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
11 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
12 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
13 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
14 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
15 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
16 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
17 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
18 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
19 

In [21]:
switch(grid_24x24,13,22)
for idx,row in enumerate(grid_24x24):
    print(idx,row)

0 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
1 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
2 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
3 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
4 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
5 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
6 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
7 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
8 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
9 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
10 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
11 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
12 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
13 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
14 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1]
15 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
16 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
17 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
18 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
19 

In [22]:
switch(grid_24x24,13,23)
for idx,row in enumerate(grid_24x24):
    print(idx,row)

0 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
1 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
2 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
3 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
4 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
5 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
6 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
7 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
8 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
9 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
10 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
11 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
12 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
13 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
14 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
15 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
16 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
17 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
18 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
19 

In [23]:
switch(grid_24x24,13,22)
for idx,row in enumerate(grid_24x24):
    print(idx,row)

0 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
1 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
2 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
3 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
4 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
5 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
6 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
7 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
8 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
9 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
10 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
11 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
12 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
13 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
14 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
15 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
16 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
17 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
18 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
19 

In [24]:
switch(grid_24x24,14,23)
for idx,row in enumerate(grid_24x24):
    print(idx,row)

0 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
1 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
2 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
3 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
4 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
5 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
6 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
7 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
8 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
9 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
10 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
11 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
12 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
13 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
14 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
15 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
16 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
17 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
18 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
19 

In [25]:
switch(grid_24x24_int,20,22)
for idx,row in enumerate(grid_24x24_int):
    print(idx,row)

0 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
1 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
2 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
3 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
4 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
5 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
6 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
7 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
8 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
9 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
10 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
11 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
12 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
13 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
14 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
15 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
16 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
17 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
18 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
19 

In [26]:
switch(grid_24x24_int,20,23)
for idx,row in enumerate(grid_24x24_int):
    print(idx,row)

0 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
1 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
2 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
3 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
4 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
5 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
6 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
7 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
8 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
9 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
10 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
11 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
12 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
13 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
14 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
15 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
16 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
17 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
18 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
19 

In [27]:
switch(grid_24x24_int,20,22)
for idx,row in enumerate(grid_24x24_int):
    print(idx,row)

0 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
1 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
2 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
3 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
4 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
5 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
6 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
7 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
8 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
9 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
10 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
11 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
12 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
13 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
14 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
15 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
16 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
17 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
18 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
19 

In [28]:
switch(grid_24x24_int,21,23)
for idx,row in enumerate(grid_24x24_int):
    print(idx,row)

0 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
1 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
2 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
3 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
4 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
5 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
6 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
7 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
8 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
9 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
10 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
11 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
12 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
13 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
14 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
15 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
16 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
17 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
18 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
19 

In [29]:
hand = [(22,3),(23,3),(22,3),(23,6),(22,8),(23,8),(22,8),(23,15),(13,22),(13,23),(13,22),(14,23),(20,22),(20,23),(20,22),(21,23)]

full = path + hand
print(full)

[(0, 0), (0, 17), (0, 6), (0, 18), (0, 8), (0, 19), (0, 10), (0, 22), (0, 11), (1, 0), (1, 12), (0, 12), (0, 14), (1, 2), (1, 17), (1, 4), (1, 18), (1, 10), (1, 23), (1, 13), (2, 0), (2, 1), (2, 2), (2, 3), (2, 10), (2, 7), (2, 12), (2, 8), (2, 22), (2, 11), (3, 3), (3, 0), (3, 4), (3, 10), (3, 5), (3, 14), (1, 14), (1, 15), (2, 15), (2, 17), (3, 7), (3, 15), (3, 11), (3, 18), (3, 13), (3, 21), (1, 21), (1, 16), (3, 17), (3, 23), (2, 23), (2, 21), (4, 4), (4, 0), (4, 5), (4, 1), (4, 9), (4, 8), (5, 0), (5, 4), (5, 3), (5, 16), (5, 6), (5, 19), (4, 19), (4, 10), (5, 7), (5, 20), (0, 20), (5, 11), (5, 21), (4, 21), (4, 15), (5, 12), (6, 6), (6, 1), (6, 7), (6, 10), (6, 9), (6, 14), (6, 11), (7, 2), (7, 0), (7, 4), (7, 1), (7, 9), (7, 3), (7, 11), (7, 12), (6, 12), (6, 20), (4, 20), (7, 13), (7, 18), (7, 15), (7, 19), (7, 16), (7, 21), (6, 21), (7, 17), (5, 17), (5, 22), (8, 1), (8, 2), (8, 17), (8, 9), (8, 19), (8, 10), (8, 23), (8, 12), (9, 1), (9, 0), (9, 13), (9, 2), (9, 16), (9, 3), 

In [31]:
for idx,(i,j) in enumerate(full):
    if grid_24x24_int[i][j]==0:
        switch(grid_24x24_int,i,j)
    else:
        print('error', idx)
        break

print(np.sum(grid_24x24_int))

576


In [8]:
full = [(0, 0), (0, 17), (0, 6), (0, 18), (0, 8), (0, 19), (0, 10), (0, 22), (0, 11), (1, 0), (1, 12), (0, 12), (0, 14), (1, 2), (1, 17), (1, 4), (1, 18), (1, 10), (1, 23), (1, 13), (2, 0), (2, 1), (2, 2), (2, 3), (2, 10), (2, 7), (2, 12), (2, 8), (2, 22), (2, 11), (3, 3), (3, 0), (3, 4), (3, 10), (3, 5), (3, 14), (1, 14), (1, 15), (2, 15), (2, 17), (3, 7), (3, 15), (3, 11), (3, 18), (3, 13), (3, 21), (1, 21), (1, 16), (3, 17), (3, 23), (2, 23), (2, 21), (4, 4), (4, 0), (4, 5), (4, 1), (4, 9), (4, 8), (5, 0), (5, 4), (5, 3), (5, 16), (5, 6), (5, 19), (4, 19), (4, 10), (5, 7), (5, 20), (0, 20), (5, 11), (5, 21), (4, 21), (4, 15), (5, 12), (6, 6), (6, 1), (6, 7), (6, 10), (6, 9), (6, 14), (6, 11), (7, 2), (7, 0), (7, 4), (7, 1), (7, 9), (7, 3), (7, 11), (7, 12), (6, 12), (6, 20), (4, 20), (7, 13), (7, 18), (7, 15), (7, 19), (7, 16), (7, 21), (6, 21), (7, 17), (5, 17), (5, 22), (8, 1), (8, 2), (8, 17), (8, 9), (8, 19), (8, 10), (8, 23), (8, 12), (9, 1), (9, 0), (9, 13), (9, 2), (9, 16), (9, 3), (9, 17), (9, 18), (9, 20), (8, 20), (8, 21), (10, 0), (10, 1), (10, 2), (10, 13), (10, 5), (10, 16), (10, 11), (10, 17), (10, 18), (10, 21), (10, 19), (11, 0), (11, 1), (11, 2), (11, 3), (11, 5), (11, 13), (11, 6), (11, 16), (11, 9), (11, 17), (11, 10), (11, 23), (9, 23), (11, 19), (12, 0), (12, 3), (12, 4), (12, 7), (12, 6), (12, 8), (12, 14), (12, 11), (12, 21), (12, 16), (12, 22), (8, 22), (13, 4), (13, 3), (13, 13), (13, 5), (13, 14), (13, 9), (14, 8), (14, 1), (14, 11), (13, 11), (13, 22), (14, 6), (14, 13), (14, 18), (14, 20), (15, 10), (15, 0), (15, 11), (15, 5), (15, 18), (15, 9), (16, 3), (16, 0), (16, 10), (16, 4), (17, 0), (17, 1), (17, 4), (17, 2), (17, 11), (17, 3), (17, 12), (17, 6), (17, 18), (17, 9), (16, 9), (16, 5), (17, 20), (17, 14), (17, 21), (18, 15), (16, 15), (18, 1), (18, 18), (18, 3), (18, 19), (18, 5), (18, 20), (18, 6), (18, 23), (10, 23), (18, 8), (19, 13), (19, 0), (19, 14), (18, 14), (18, 22), (19, 1), (19, 17), (19, 3), (19, 18), (19, 4), (20, 0), (20, 7), (20, 2), (20, 9), (20, 3), (20, 10), (19, 10), (19, 6), (20, 4), (20, 16), (20, 14), (20, 17), (20, 19), (20, 22), (19, 22), (19, 11), (21, 9), (21, 0), (21, 11), (21, 5), (21, 16), (21, 8), (21, 17), (21, 10), (22, 1), (22, 2), (22, 7), (22, 3), (22, 10), (22, 4), (23, 0), (23, 13), (22, 13), (22, 12), (21, 12), (21, 14), (22, 14), (22, 15), (23, 2), (23, 18), (21, 18), (21, 22), (22, 22), (23, 20), (22, 21), (19, 21), (23, 19), (23, 4), (23, 23), (12, 23), (23, 5), (22, 19), (22, 3), (23, 3), (22, 3), (23, 6), (22, 8), (23, 8), (22, 8), (23, 15), (13, 22), (13, 23), (13, 22), (14, 23), (20, 22), (20, 23), (20, 22), (21, 23)]
submission = [changeIndex(i,j,24) for (i,j) in full]
print(submission)

[(1, 24), (18, 24), (7, 24), (19, 24), (9, 24), (20, 24), (11, 24), (23, 24), (12, 24), (1, 23), (13, 23), (13, 24), (15, 24), (3, 23), (18, 23), (5, 23), (19, 23), (11, 23), (24, 23), (14, 23), (1, 22), (2, 22), (3, 22), (4, 22), (11, 22), (8, 22), (13, 22), (9, 22), (23, 22), (12, 22), (4, 21), (1, 21), (5, 21), (11, 21), (6, 21), (15, 21), (15, 23), (16, 23), (16, 22), (18, 22), (8, 21), (16, 21), (12, 21), (19, 21), (14, 21), (22, 21), (22, 23), (17, 23), (18, 21), (24, 21), (24, 22), (22, 22), (5, 20), (1, 20), (6, 20), (2, 20), (10, 20), (9, 20), (1, 19), (5, 19), (4, 19), (17, 19), (7, 19), (20, 19), (20, 20), (11, 20), (8, 19), (21, 19), (21, 24), (12, 19), (22, 19), (22, 20), (16, 20), (13, 19), (7, 18), (2, 18), (8, 18), (11, 18), (10, 18), (15, 18), (12, 18), (3, 17), (1, 17), (5, 17), (2, 17), (10, 17), (4, 17), (12, 17), (13, 17), (13, 18), (21, 18), (21, 20), (14, 17), (19, 17), (16, 17), (20, 17), (17, 17), (22, 17), (22, 18), (18, 17), (18, 19), (23, 19), (2, 16), (3, 1

In [11]:
print(np.sum(grid_24x24))

for (i,j) in full:
    if grid_24x24[i][j]==1:
        print('failed!!!')
        break
    else:
        switch(grid_24x24,i,j)

print(np.sum(grid_24x24))
for row in grid_24x24:
    print(row)

260
576
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

In [32]:
print(submission)

288

In [46]:
submission = [changeIndex(i,j,24) for (i,j) in full]
submission

[(1, 24),
 (18, 24),
 (7, 24),
 (19, 24),
 (9, 24),
 (20, 24),
 (11, 24),
 (23, 24),
 (12, 24),
 (1, 23),
 (13, 23),
 (13, 24),
 (15, 24),
 (3, 23),
 (18, 23),
 (5, 23),
 (19, 23),
 (11, 23),
 (24, 23),
 (14, 23),
 (1, 22),
 (2, 22),
 (3, 22),
 (4, 22),
 (11, 22),
 (8, 22),
 (13, 22),
 (9, 22),
 (23, 22),
 (12, 22),
 (4, 21),
 (1, 21),
 (5, 21),
 (11, 21),
 (6, 21),
 (15, 21),
 (15, 23),
 (16, 23),
 (16, 22),
 (18, 22),
 (8, 21),
 (16, 21),
 (12, 21),
 (19, 21),
 (14, 21),
 (22, 21),
 (22, 23),
 (17, 23),
 (18, 21),
 (24, 21),
 (24, 22),
 (22, 22),
 (5, 20),
 (1, 20),
 (6, 20),
 (2, 20),
 (10, 20),
 (9, 20),
 (1, 19),
 (5, 19),
 (4, 19),
 (17, 19),
 (7, 19),
 (20, 19),
 (20, 20),
 (11, 20),
 (8, 19),
 (21, 19),
 (21, 24),
 (12, 19),
 (22, 19),
 (22, 20),
 (16, 20),
 (13, 19),
 (7, 18),
 (2, 18),
 (8, 18),
 (11, 18),
 (10, 18),
 (15, 18),
 (12, 18),
 (3, 17),
 (1, 17),
 (5, 17),
 (2, 17),
 (10, 17),
 (4, 17),
 (12, 17),
 (13, 17),
 (13, 18),
 (21, 18),
 (21, 20),
 (14, 17),
 (19, 17),
 

In [2]:
import numpy as np
grid_30x30 = '''
    110001000000100101110011001000
    010100001100011101010111100110
    000110011010011111100010100010
    111101110110011101110100110001
    000110001000100011001101100010
    101111001110110010111101001111
    001110000101101001101000001101
    111001110000101011111111110100
    110000000000110111111001100100
    111001110100111110001110111011
    111010100010010100000001101100
    010111110011001111110100001001
    010100111011000001100000011010
    010001010110111100100111001101
    111111010001011100101100110110
    101000110110010111111011001001
    111011000100101111101001100010
    101001100011010100010000100001
    111111100111111110010111110010
    010000010000011001001010010011
    111010110011011111101100110110
    011100110001101001100000000110
    111110100101010000100011011010
    111100011111000011110001001111
    111000111111101011111011100100
    101011000011001110101011000011
    001101011101000001100101101001
    010010100000011011100101010001
    010111101001110100010110010010
    110000011010111110100110000010
    '''

rows = grid_30x30.strip().split('\n')
grid_30x30 = np.array([list(map(int, row.strip())) for row in rows], dtype=int)

In [7]:
s_30x30 = lights_out(grid_30x30)

In [8]:
for row in s_30x30:
    print('{',end='')
    for n in row:
        print(str(n)+',',end='')
    print('},')

{1,1,0,1,0,1,1,0,0,1,1,0,1,0,0,1,0,0,1,0,1,1,1,0,1,1,1,0,1,1,},
{0,1,0,0,0,0,1,0,1,0,1,0,0,1,1,1,0,0,0,0,1,0,1,0,0,1,0,1,0,1,},
{1,1,1,1,0,1,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1,1,0,},
{1,1,1,0,0,1,0,1,0,0,0,0,0,1,1,1,0,0,1,0,1,0,0,1,0,0,0,0,1,0,},
{0,0,0,0,1,0,1,0,1,1,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,},
{0,1,0,1,0,0,0,1,0,1,1,1,0,0,1,1,0,0,0,1,1,1,1,1,0,0,0,0,1,1,},
{0,0,1,0,1,0,1,0,0,0,1,1,1,0,1,0,0,0,1,1,0,1,0,1,1,1,1,1,1,0,},
{1,1,1,1,0,1,0,1,0,1,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,1,1,},
{1,1,0,1,0,0,1,0,0,1,1,0,1,1,0,1,1,0,1,0,0,1,0,0,0,1,0,1,1,1,},
{0,0,0,0,1,0,1,0,1,1,0,1,0,0,0,0,0,0,1,0,1,1,0,0,1,1,0,1,1,1,},
{0,0,0,0,0,1,1,1,1,0,1,1,1,0,1,0,1,0,1,0,0,0,1,1,1,0,0,0,0,0,},
{0,1,0,0,1,1,0,1,0,1,0,1,0,0,1,1,1,0,1,0,1,0,0,1,1,1,1,0,1,0,},
{1,0,1,1,1,1,1,0,0,0,1,0,1,1,1,1,1,1,0,0,0,0,1,0,0,1,0,1,1,0,},
{1,0,1,0,1,0,0,0,1,1,1,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,},
{0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,1,1,0,1,1,1,0,1,0,},
{1,0,1,1,0,0,0,1,0,0,0,0,0,1,0,1,1,0,1,0

In [13]:
for (i,j),e in np.ndenumerate(solution):
    if e and (i,j) not in path:
        print(i,j)

13 23
14 23
20 23
21 23
23 3
23 6
23 8
23 15
